# Optimizing Rosenbrock's function using vanilla BFGS algorithm

In [2]:
using ForwardDiff
using LinearAlgebra

### Rosenbrock's Function

In [3]:
rosenbrock(Θ) = sum([100*(Θ[i+1]-Θ[i]^2)^2 + (1-Θ[i])^2 for i in 1:(length(Θ)-1)]);

In [4]:
rosenbrock_gradient(Θ) = ForwardDiff.gradient(rosenbrock, Θ);

### BFGS Algorithm

In [5]:
function rosenbrock_BFGS( Θ, ITERS )
    
    DIM = length(Θ)
    Q = Matrix(1.0I, DIM, DIM)
    arguments = zeros(DIM, ITERS)
    values = []
    
    for i in 1:ITERS
        
        # History
        arguments[1,i] = round( Θ[1], digits=8 )
        arguments[2,i] = round( Θ[2], digits=8 )
        push!(values, round(rosenbrock(Θ), digits=8))
        
        # Finding new weights and update
        Θ, Q = BFGS( Θ, Q )
    
    end
    return arguments, values, Q
end;

In [6]:
function BFGS( Θ, Q ) # author: Bartosz Chaber
    
    f = rosenbrock
    ∇f = rosenbrock_gradient
    g = ∇f(Θ)
    
    # Looking for the best step size
    d  = -Q*g    # Direction
    ϕ  = α -> f(Θ + α*d)
    ϕ′ = α -> ∇f(Θ + α*d) ⋅ d
    α = line_search( ϕ, ϕ′, d )  #  α is not a hyperparameter!
    
    # New weights
    Θ′ = Θ + α * d
    
    # Hessian Approximation Update
    g′ = ∇f( Θ′ )
    δ = Θ′ - Θ
    γ = g′ - g    
    Q′ = Q - (δ*γ'*Q + Q*γ*δ') / (δ'*γ) +
                 (1.0 + (γ'*Q*γ) / (δ'*γ)) *
                          (δ*δ') / (δ'*γ)
    
    return Θ′, Q′

end;

In [7]:
function line_search( ϕ, ϕ′, d ) # author: Bartosz Chaber
    a, b = bracket_minimum(ϕ)
    x, y = golden_section_search(ϕ, a, b)
    x/2 + y/2
end;

In [8]:
function bracket_minimum(f; x=0, s=1e-2, k=2.0) # author: the book
    a, ya = x, f(x)
    b, yb = a + s, f(a + s)
    if yb > ya
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    end
    while true
        c, yc = b + s, f(b + s)
        if yc > yb
            return a < c ? (a, c) : (c, a)
        end
        a, ya, b, yb = b, yb, c, yc
        s *= k
    end
end;

In [9]:
import Base.MathConstants: φ

function golden_section_search( f, a, b; n=50 ) # author: the book
    ρ = φ-1
    d = ρ * b + (1 - ρ)*a
    yd = f(d)
    for i = 1 : n-1
        c = ρ*a + (1 - ρ)*b
        yc = f(c)
        if yc < yd
            b, d, yd = d, c, yc
        else
            a, b = b, c
        end
    end
    return a < b ? (a, b) : (b, a)
end;

### BFGS Training

In [10]:
ITERS = 10;
Θ = [rand(), rand()];

In [11]:
arguments, values, Q = rosenbrock_BFGS( Θ, ITERS );

In [12]:
println(values)
arguments

Any[0.6201468, 0.20158248, 0.1527435, 0.0892933, 0.05426181, 0.02812169, 0.01166783, 0.00460196, 0.00097647, 5.826e-5]


2×10 Array{Float64,2}:
 0.518758  0.551581  0.64024   0.758301  …  0.932179  0.978145  0.993574
 0.331444  0.301999  0.394638  0.557449     0.86911   0.954534  0.987601

### Visualization

In [ ]:
using NBInclude
@nbinclude("Visualization.ipynb");

┌ Info: Precompiling Gadfly [c91e804a-d5a3-530f-b6f0-dfbca275c004]
└ @ Base loading.jl:1278


In [ ]:
y = [round(v, digits=8) for v in values]
visualize_training_process( length(y), y,
    "BFGS training", "Iteration", "Rosenbrock's function value" )

In [ ]:
SPAN = 100; LEFT = 0; RIGHT = 1.5
ax = LinRange( LEFT, RIGHT, SPAN );

levels = [LinRange(0, 100, 20)...];

In [ ]:
contour_cost(
    ax, ax,
    (x, y) -> rosenbrock( [x, y] ), levels,
    arguments, 1,
    "Optimization Rosenbrock's function with two arguments (BFGS)",
    "Θ1", "Θ2",
)

In [ ]:
arguments[:, end] # final (best) weights